<a href="https://colab.research.google.com/github/skimaza/assist_ai/blob/main/cnn_models_detectron2_assist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝의 이해
# CNN 모델 - Facebook Detectron2 예제
## Detectron2 소스 : https://github.com/facebookresearch/detectron2
## https://towardsdatascience.com/a-beginners-guide-to-object-detection-and-computer-vision-with-facebook-s-detectron2-700b6273390e 예제를 강좌에 맞춰 수정

# MS COCO dataset 을 이용하기 위한 코드 설치
MS COCO는 마이크로소프트에서 만든 객체 인식, 탐지, 세그멘테이션 데이터셋  
Common Objects in Context (https://cocodataset.org)  
객체 클래스는 80개

Colab에 기본으로 포함되지 않은 패키지는 shell command로 설치해야 함  
pip (Preferred Installer Program)  
파이썬 패키지에 포함된 기본 설치 프로그램

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip uninstall flask -y

In [ ]:
#!python -m pip install pyyaml==5.1
!python -m pip install pyyaml==5.3.1


In [ ]:
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'


In [ ]:
dist = distutils.core.run_setup("./detectron2/setup.py")


In [ ]:
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}


In [ ]:
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
#!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

## Detectron2 설치


In [ ]:
#!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# 필요한 패키지 import

In [ ]:
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow # On your local machine you don’t need it.
import json
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from google.colab.patches import cv2_imshow

**from google.colab.patches import cv2_imshow**  

OpenCV의 이미지 디스플레이 함수인 cv2.imshow() 함수가 colab과 호환되지 않음.  
Google에서 cv2_imshow라는 함수를 정의해서 동일한 기능을 제공하고 있음  
(주의) '.' 대신 '_'로 연결되어 있음

# 샘플이미지 다운로드

인터넷 URL을 로컬 컴퓨터에 다운로드할 때는 wget 명령 사용  
-O 옵션으로 저장할 파일이름 지정

In [ ]:
!wget https://miro.medium.com/max/2000/1*sxjvURCdHBC0MkuZkPnIkQ.png -O street.png

In [ ]:
im = cv2.imread('street.png')
cv2_imshow(im)

# Configuration 확인

In [ ]:
cfg = get_cfg()

In [ ]:
#cfg

In [ ]:
cfg.VERSION

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # set threshold for this model

In [ ]:
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml')

In [ ]:
cfg.MODEL.DEVICE

In [ ]:
cfg.MODEL.META_ARCHITECTURE

# GPU 사용 여부에 따라 configuration이 달라짐
## Colab에서 GPU 할당받았는지 확인하고 configuration을 맞춰줌
## 디폴트는 DEVICE가 'gpu'로 설정되어 있음

In [ ]:
import os

def check_cpu_mode():
    if (int(os.environ["COLAB_GPU"]) > 0):
        return False
    elif ("COLAB_TPU_ADDR" in os.environ and os.environ["COLAB_TPU_ADDR"]):
        return False
    else:
        return True

In [ ]:
if check_cpu_mode():
    cfg.MODEL.DEVICE = 'cpu'

In [ ]:
cfg.MODEL.DEVICE

In [ ]:
predictor = DefaultPredictor(cfg)

# predictor model의 구성 살펴보기

In [ ]:
predictor.model

In [ ]:
predictor.model.backbone

In [ ]:
predictor.model.proposal_generator

In [ ]:
predictor.model.roi_heads

# 이미지의 색상 포맷
## 디폴트가 BGR임

In [ ]:
cfg.INPUT.FORMAT

# 이미지에서 객체 탐지

In [ ]:
outputs = predictor(im)

In [ ]:
outputs.keys()

# 결과부터 보기

In [ ]:
v = Visualizer(im, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
v = v.draw_instance_predictions(outputs['instances'].to('cpu'))
cv2_imshow(v.get_image())

# 탐지된 객체 리스트

In [ ]:
instances = outputs['instances']

In [ ]:
len(instances)

In [ ]:
instances[0]

In [ ]:
instances[0].pred_masks.shape

In [ ]:
instances[0].pred_masks.nonzero()

첫번 인스턴스의 영역중 일부인 [513:583, 637:654] 의 이미지 확인

In [ ]:
cv2_imshow(im[510:590, 630:660])

## pred_masks는 탐지된 객체의 픽셀

In [ ]:
instances.pred_classes, instances.pred_classes.size()

31개의 객체 탐지. pred_classes는 각 인스턴스의 클래스. 즉 첫번째 인스턴스는 2번 클래스, 두번째 인스턴스는 0번 클래스, ...

In [ ]:
instances.scores, instances.scores.size()

# 객체 클래스 확인

In [ ]:
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
metadata

In [ ]:
class_catalog = metadata.thing_classes

In [ ]:
class_catalog[2]

In [ ]:
class_catalog[0]

In [ ]:
class_catalog[7]

In [ ]:
class_catalog[3]

In [ ]:
len(class_catalog)

In [ ]:
instances.pred_boxes, len(instances.pred_boxes)

첫번째 박스 영역 보기

In [ ]:
instances.pred_boxes[0]

위 결과는 graphics 좌표계임에 주의 (왼쪽 위가 0,0이고 x축이 수평, y축이 수직).  
numpy image array는 row가 수직, column이 수평 좌표.  
따라서 첫번 좌표 (604, 511)는 image에서는 im[511, 604]임  
첫번 box의 numpy array상의 데이터는 [511:584, 604:702]에 해당.  

In [ ]:
cv2_imshow(im[511:584, 604:703])

전체 결과 표시는 Visualizer를 사용하여 확인

In [ ]:
v = Visualizer(im, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
v = v.draw_instance_predictions(outputs['instances'].to('cpu'))
cv2_imshow(v.get_image())

# 다른 이미지로 테스트

https://nypost.com/wp-content/uploads/sites/2/2017/12/nyc-streets.jpg

In [ ]:
!wget https://nypost.com/wp-content/uploads/sites/2/2017/12/nyc-streets.jpg -O street2.jpg

In [ ]:
!ls -l

In [ ]:
im2 = cv2.imread('street2.jpg')
cv2_imshow(im2)

In [ ]:
outputs2 = predictor(im2)
instances2 = outputs2['instances']

v2 = Visualizer(im2, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
v2 = v2.draw_instance_predictions(outputs2['instances'].to('cpu'))
cv2_imshow(v2.get_image())

https://media.timeout.com/images/102973595/750/422/image.jpg

In [ ]:
!wget https://media.timeout.com/images/102973595/750/422/image.jpg -O living_room1.jpg

In [ ]:
!ls -l

In [ ]:
im2 = cv2.imread('living_room1.jpg')
cv2_imshow(im2)

In [ ]:
outputs2 = predictor(im2)
instances2 = outputs2['instances']

v2 = Visualizer(im2, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
v2 = v2.draw_instance_predictions(outputs2['instances'].to('cpu'))
cv2_imshow(v2.get_image())

In [ ]:
# v2 = Visualizer(im2, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.5)
# v2 = v2.draw_instance_predictions(outputs2['instances'].to('cpu'))
# cv2_imshow(v2.get_image())